In [ ]:
using JLD2
using PyPlot
using StatsBase # Statistics
using Distributions

using ScikitLearn # machine learning package
@sk_import gaussian_process : GaussianProcessRegressor
@sk_import gaussian_process.kernels : Matern

using PyCall
@pyimport matplotlib.animation as anim

# config plot settings
PyPlot.matplotlib.style.use("ggplot")
rcParams = PyPlot.PyDict(PyPlot.matplotlib."rcParams")
rcParams["font.size"] = 16;

In [ ]:
@load joinpath(pwd(), "targets_and_normalized_features.jld2") X henry_y gcmc_y